# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Лабораторная работа 6.1

### Форматирование строк

6\.1\.1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [1]:
import pandas as pd

In [2]:
recipes = pd.read_csv('recipes_sample.csv', delimiter=',')
recipes_random = recipes.sample(n = 5)
recipes_random = recipes_random[['id', 'minutes']]
recipes_id = recipes_random['id'].tolist()
recipes_minutes = recipes_random['minutes'].tolist()
print(f'|      id      |  minutes  |')
print('|--------------------------|')
for index in range(5):
    print(f"|{recipes_id[index]: ^14}|{recipes_minutes[index]: ^11}|")

|      id      |  minutes  |
|--------------------------|
|    387400    |    40     |
|    193733    |    15     |
|     2974     |    115    |
|    171348    |     5     |
|    107418    |     6     |


6\.1\.2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [3]:
import csv
import xml.etree.ElementTree as ET
def show_info(recipe_id):
    with open('recipes_sample.csv', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['id'] == str(recipe_id):
                name = row['name'].strip()
                author_id = row['contributor_id']
                minutes = row['minutes']
                break
    tree = ET.parse('steps_sample.xml')
    root = tree.getroot()
    steps = [step.text.strip() for step in root.findall(f"./recipe[@id='{recipe_id}']/steps/step")]
    string = f'"{name.title()}"\n\n'
    for i, step in enumerate(steps, start=1):
        string += f'{i}. {step}\n'
    string += '------------------------------\n'
    string += f'Автор: {author_id}\n'
    string += f'Среднее время приготовления: {minutes} минут\n'
    return string
print(show_info(170895))

"Leeks And Parsnips  Sauteed Or Creamed"

------------------------------
Автор: 8377
Среднее время приготовления: 27 минут



## Работа с регулярными выражениями

In [4]:
import re

In [5]:
from bs4 import BeautifulSoup
with open('steps_sample.xml') as f:
    rb = BeautifulSoup(f, 'xml')
res = list()
for recipee in rb.recipes.find_all('recipe'):
    st = [steps.next for steps in recipee.steps.find_all('step')]
    res.append({recipee.find('id').next: st})
dct = res[0]
for index in range(1, len(res)):
    dct.update(res[index])
dct

{'44123': ['in 1 / 4 cup butter , saute carrots , onion , celery and broccoli stems for 5 minutes',
  'add thyme , oregano and basil',
  'saute 5 minutes more',
  'add wine and deglaze pan',
  'add hot chicken stock and reduce by one-third',
  'add worcestershire sauce , tabasco , smoked chicken , beans and broccoli florets',
  'simmer 5 minutes',
  'add cream , simmer 5 minutes more and season to taste',
  'drop in remaining butter , piece by piece , stirring until melted and serve immediately',
  'smoked chicken: on a covered grill , slightly smoke boneless chicken , cooking to medium rare',
  'chef meskan uses applewood chips and does not allow the grill to become too hot'],
 '67664': ['mix all the ingredients using a blender',
  'pour into popsicle molds',
  'freeze and enjoy !'],
 '38798': ['combine all ingredients in a large bowl and mix well',
  'shape into one-inch balls',
  'cover and refrigerate or freeze until ready to bake',
  'preheat oven to 350 degrees',
  'place on ungr

6\.1\.3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [6]:
steps = dct['25082']
steps

['proof yeast in half a cup of the water , with about a teaspoon of flour- let it sit , covered , until it becomes bubbly& active',
 'combine the rest of the water with the yeast / water in a large bowl , and gradually add about half of the flour',
 'stir the mixture about 100 times in the same direction , until it is well-mixed and strands are beginning to form',
 'stir in the salt and oil',
 'add the remaining flour half a cup at a time , stirring well , until it is too stiff to stir with a spoon',
 'turn out onto a lightly floured board and knead for about 20 minutes , adding flour as nescessary to keep the dough from sticking to the board',
 'the dough should be springy and nice to work with',
 'when it has been sufficiently kneaded , cover it with a damp cloth for about 10 minutes and wash and grease the bowl lightly',
 'after it has rested , knead the dough a few more times and place it in the bowl , covered with the damp cloth , a plastic bag , and maybe a plate',
 'let the doug

In [7]:
steps = dct['25082']
p = re.compile(r'\d+\s+(hours?|minutes?)')
for step in steps:
    match = re.search(p, step)
    if match:
        print(match.group())

20 minutes
10 minutes
2 hours
10 minutes
20 minutes


6\.1\.4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [8]:
p = re.compile(r'[^this]\w, but') 
strings = ['this takes about, but 2 hours', 'stir in the salt and oil', 'work with but']
for el in strings:
    if re.search(p, el):
        print(el)

this takes about, but 2 hours


## Лабораторная 6.2

6\.2\.1\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [9]:
recipes = pd.read_csv('recipes_sample.csv')
recipe_id = 72367
recipe_steps = recipes.loc[recipes['id'] == recipe_id, 'description'].iloc[0]
recipe_steps = re.sub(r'\s*/\s*', '/', recipe_steps)
(recipe_steps)

'such a beautiful tart - just like from a european bakery, and delicious to boot! from worldwiderecipes i believe.'

In [10]:
recipe_steps = [' 1 / 2 cup plus 2 tablespoons sugar', 'combine the rest of the water with the yeast / water in a large bowl', 'combine mayo']
recipe_steps = [re.sub(r'\s*/\s*', '/', step) for step in recipe_steps]
print(recipe_steps)

[' 1/2 cup plus 2 tablespoons sugar', 'combine the rest of the water with the yeast/water in a large bowl', 'combine mayo']


### Сегментация текста

In [11]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [13]:
from nltk.tokenize import word_tokenize

6\.2\.2\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [14]:
recipes = pd.read_csv('recipes_sample.csv')
words = []
for steps in recipes['description']:
    if isinstance(steps, str):
        words += nltk.word_tokenize(steps)
unique_words = set([word.lower() for word in words if word.isalpha()])
print(len(unique_words))
print(unique_words)

22049
{'clippng', 'mountain', 'foods', 'freezed', 'catalonia', 'rounds', 'coarser', 'heartburn', 'demonstrate', 'smooooth', 'guadalajara', 'terms', 'overwelm', 'figuring', 'berggarten', 'signed', 'motel', 'cristobal', 'lunacity', 'segal', 'romanesco', 'gugelhupf', 'bisques', 'terraines', 'ick', 'colbin', 'flava', 'textural', 'karpenko', 'accentuate', 'patient', 'prior', 'crackers', 'cripsy', 'bureau', 'landed', 'shopper', 'concord', 'gras', 'sharon', 'fire', 'fiance', 'nervous', 'sandwichs', 'glory', 'burek', 'freeform', 'cafix', 'bag', 'remain', 'mccartney', 'excluding', 'beer', 'blackheads', 'lattes', 'cousins', 'jayson', 'kashmiri', 'stripes', 'van', 'midget', 'buddha', 'reccomended', 'website', 'rouge', 'flatter', 'baptisms', 'prebrowning', 'croissants', 'mousse', 'expandex', 'honey', 'products', 'tongue', 'anybody', 'fes', 'buster', 'haul', 'barilla', 'cooklikeachampion', 'balmy', 'bhat', 'visible', 'fumé', 'wavy', 'happen', 'dunkables', 'bev', 'divvy', 'rut', 'requested', 'overhe

6\.2\.3\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [15]:
recipes = pd.read_csv('recipes_sample.csv')
recipes['description'] = recipes['description'].fillna('')
from nltk.tokenize import sent_tokenize
recipes['sentences'] = recipes['description'].apply(lambda x: sent_tokenize(x, language='english'))
top_5_recipes = recipes.sort_values(by='sentences', ascending=False).head(5)
for i, row in top_5_recipes.iterrows():
    print(f"Recipe {i+1} ({len(row['sentences'])} sentences):\n{row['description']}\n")

Recipe 29729 (6 sentences):
“ready, set, cook! hidden valley contest entry” friends and family request my yummy 7 layer salad recipe #51349. now i just made it yummier!  a party salad thats crisp, crunchy and refreshing. make the day before for ease of the party day. travels well so make it, bring it and watch it disappear.  recipe #107440 is an easy method and tasty on the salad.

Recipe 22142 (2 sentences):
“ready, set, cook! hidden valley contest entry”

Recipe 26370 (5 sentences):
“ragú® recipe contest entry". do you want a quick, easy and tasty brunch, lunch or dinner?  grab a jar of ragu sauce, add chickpeas and eggs, top it with feta cheese and pine nuts and you have a super easy and satifying meal in no time. oh! and don't forget some crusty bread to soak up all that delicious tomato sauce!

Recipe 4478 (9 sentences):
“how’s my peanut butter sandwich coming?  you’re using chunky peanut butter, right?  i won’t eat smooth!  make it an open face sandwich, too!  don’t put any jelly

In [16]:
from nltk import word_tokenize
df = pd.DataFrame(recipes, columns=['description'])
df.head()
txt = df.description.apply(word_tokenize)
txt.head()

0    [an, original, recipe, created, by, chef, scot...
1    [my, children, and, their, friends, ask, for, ...
2    [these, were, so, go, ,, it, surprised, even, ...
3    [my, sister-in-law, made, these, for, us, at, ...
4    [i, think, a, fondue, is, a, very, romantic, c...
Name: description, dtype: object

6\.2\.4\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [17]:
import nltk
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tyute\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [18]:
def print_pos_tags(sentence):
    words = word_tokenize(sentence)
    pos_tags = nltk.pos_tag(words)
    for tag, word in pos_tags:
        print(f"{tag:^7}", end=" ")
    print()
    for tag, word in pos_tags:
        print(f"{word:^7}", end=" ")
df = recipes[recipes['id'] == 241106]
recipe_name = df['name'].values[0]
print_pos_tags(recipe_name)

eggplant steaks   with   chickpeas  feta   cheese    and    black  olives  
  JJ      NNS     IN      NNS     VBP     JJ      CC      JJ      NNS   